The following code runs a simulation of diffusion on a lattice with constant grid spacing, with equal probability of taking a step in every direction with every time step. The grid spacing is $\Delta = \sigma\sqrt{\tau}=\sqrt{4D\tau}$, where $D$ is the diffusion constant, $\sigma$ is the variance and $\tau$ is the time step

Individual molecules are allowed to diffuse on a grid and bleach. To deal with photobleaching, I am assuming that every fluorophore emits the same number of photons per frame, and then calculating the probability of bleaching for every frame with a Monte Carlo algorithm.

In [ ]:
# Main simulation routine
import photogate as pg
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import csv
import os
%matplotlib

# for debugging only: 
%load_ext autoreload  
%autoreload 2

### Input user-defined parameters #########################################

plot_while_simulating = False
write_file_name, save_to_file = 'data', True
calculate_diffusion_const = False

# Override any default simulation and fluorescence parameters
sim_params = {'num_particles':80000, 
              'sim_time' : 80,
              'frame_rate' : 10,
              'diffusion_const' : 0.5,
              'num_time_steps' : 20000,
              'tracking_roi' : 7}
fluo_params = {'num_steps':50, 
               'intensity_tirf':2000, 
               'r_ring':8, 'r_tirf':7} # 7
timing_params = {'gate_freq' : 3} # 0.5


#ints = np.arange(0,60001,4000)
#dcs = np.arange(0.1,0.11,0.05) #np.arange(0.01,4,0.1)
#gate_freqs = np.arange(0.05,6,0.2)
data_dir = os.path.join('output','test')
for intensity in range(5):
    
    print('starting loop for trial '+str(intensity))
    write_file_name = 'data_trial_'+str(intensity)
    #sim_params['diffusion_const'] = dc
    #fluo_params['intensity_tirf'] = intensity
    
    ### Set up the simulation ############################################## 

    cell = pg.DiffusionSimulation(**sim_params)

    gate_timing, p_bleach_inv, fluo_profiles = pg.init_gate_timers(cell.tau, 
                                    fluo_params=fluo_params, **timing_params)

    p_bleach_inv_tirf = fluo_profiles[0].tirf_bleaching_profile(cell.tau)

    if plot_while_simulating:
        fig = plt.figure("live_plot")
        plt.ion()
    else:
        plt.ioff()

    if save_to_file:
        file_writer, data_file, dirs = pg.init_save_file(cell, 
                fluo_profiles[-1], gate_timing, write_file_name, data_dir)

    ### Set up arrays to store plotting parameters ###########################  
    fluo_density_in_roi = np.zeros(cell.num_time_steps)
    percent_oligomers = np.zeros(cell.num_time_steps)

    if calculate_diffusion_const:
        initial_positions = np.copy(cell.particles[:,0:2])
        mean_x_sq = np.zeros(cell.num_time_steps)

    ### Run the simulation ###################################################
    start_time = time.time()

    for i in range(0, cell.num_time_steps):

        # Update the bleaching pattern depending on point in the cycle
        gate_or_tirf = pg.gate_update(gate_timing, cell.tau, 
                                    p_bleach_inv, fluo_profiles) 
        # Move and bleach particles
        cell.move_particles() 
        cell.bleach_particles(p_bleach_inv, p_bleach_inv_tirf, gate_or_tirf)

        # Calculate any required metrics
        fluo_density_in_roi[i], percent_oligomers[i] = \
            cell.unbleached_particle_density()
        cell.update_tracking_history(i)
        
        if calculate_diffusion_const:     
            displacements_sq = np.square((cell.particles[:,0:2] - \
                        initial_positions[:,:])  * cell.grid_delta)
            mean_x_sq[i] = np.mean(displacements_sq)

        # Save data
        if (i%cell.steps_per_frame == 0) and save_to_file:     
            pg.write_positions_to_file(cell, gate_or_tirf, file_writer)

        # Plot data if required
        if (i%50 == 0) and plot_while_simulating:
            pg.plot_current_state(cell, plot_trackable=True)
            plt.pause(0.0001)
            plt.clf()    

        if i%1000 == 0:
            print('step '+str(i)+' of '+str(cell.num_time_steps))

    plt.close('all')
    plt.pause(0.1)
    plt.ioff()

    fig1 = pg.plot_fluo_density_results(cell, fluo_density_in_roi, 
                                        percent_oligomers)
    fig2 = pg.plot_tracking_results(cell)

    if save_to_file:
        data_file.close()
        fig1.savefig(os.path.join(dirs['png'], write_file_name+'.png'))
        fig1.savefig(os.path.join(dirs['svg'], write_file_name+'.svg'))
        fig2.savefig(os.path.join(dirs['png'], write_file_name+'_track.png'))
        fig2.savefig(os.path.join(dirs['svg'], write_file_name+'_track.svg'))
        np.savetxt(os.path.join(dirs['results'], 
                write_file_name+'_density.txt'), fluo_density_in_roi)
        np.savetxt(os.path.join(dirs['results'], 
                write_file_name+'_percent_oligo.txt'), percent_oligomers)
        
        f = os.path.join(dirs['results'], 
                         write_file_name+'_tracking_times.csv')
        pg.write_tracking_times_csv(cell, f)
    else:
        plt.show()

    if calculate_diffusion_const:
        plt.figure('diffusion constant')
        time_vect = np.linspace(0,cell.sim_time, num=cell.num_time_steps)
        plt.plot(time_vect[1:], np.divide(mean_x_sq[1:], (4 * time_vect[1:])))
        plt.show()
    
    print("Finished simulation in", time.time() - start_time, "seconds")



In [ ]:
""" Plot data from tracking_times with statistics """
import matplotlib.pyplot as plt
import photogate as pg
import numpy as np
from scipy.stats import expon
import csv
import os
# for debugging only: 
%load_ext autoreload  
%autoreload 2

#base_dir = 'E:\\Work\\Coding\\Python\\PhotoGateSimulation\\output'
base_dir = 'E:\\Work\\Data\\Simulation results\\2D photogate\\WITH TRACKING DATA'
#target_dirs = ['2000_int_TIRF_TOCCSL_trials',
#               '2000_int_TIRF_gate_trials']
#target_dirs = ['20000_TIRF_TOCCSL_10_trials',
#               '20000_TIRF_gate_10_trials']
target_dirs = ['05D_TIRF_gate_trials',
              '05D_TIRF_TOCCSL_trials']
min_traject_time = 0.5 # trajectories shorter than this will be discarded

track_times =[]

for curr_dir in target_dirs:
    target_dir = os.path.join(base_dir, curr_dir)
    result_dir = os.path.join(target_dir, 'data', 'results')

    dirs = {'results' : os.path.join(target_dir, 'data', 'results'),
            'inf' : os.path.join(target_dir, 'data', 'inf'),
            'save' : os.path.join(target_dir, 'figs', 'analysis')}

    if not os.path.exists(dirs['save']) : os.makedirs(dirs['save'])

    all_results = os.listdir(dirs['results'])

    result_names = [x for x in all_results if '_tracking_times' in x]
    result_files = [os.path.join(dirs['results'], x) for x in result_names]

    data = pg.load_tracking_results(result_files)

    fig, all_tracks, bleach_tracks = pg.plot_tracking_statistics(data,
                                        print_results=True)
    track_times.append(all_tracks)

fig2, ax2 = plt.subplots(nrows=1, ncols=1)

max_time = max([max(x) for x in track_times])
means = [np.mean(x) for x in track_times]
print(means)

track_times_trim = [[t for t in x if t>min_traject_time] for x in track_times]
bins = np.arange(0,max_time, 0.5)

bin_vals = ax2.hist(track_times_trim, bins, histtype='bar', 
                    normed=False, label=target_dirs, log=False)

ax2.legend(prop={'size': 10})
ax2.set_xlim([0,round(max_time+1)])
ax2.set_xlabel('Tracking time (s)')
ax2.set_ylabel('Fraction of trajectories')

#print(bins)
#print(bin_vals)

#ax2.set_yscale('log')

#a = expon.fit(all_tracks)
#print(a)
#mean = expon.mean(a)
#mean2, std2 = np.mean(all_tracks), np.std(all_tracks)
#std = expon.std(a)
#print(a)
#print(mean2, std2)
#print(expon.fit.__doc__)
plt.show()

In [ ]:
import numpy as np
a = ([np.array([   0.,  383.,  250.,  130.,   96.,   68.,   49.,   45.,   30.,
         30.,   14.,    7.,   11.,   10.,    4.,    8.,    8.,    3.,
          4.,    4.,    7.,    2.,    2.,    1.,    1.,    1.,    2.,
          1.,    2.,    1.,    0.,    0.,    1.,    0.,    1.,    0.,    1.]), 
np.array([  0.,  66.,  16.,   8.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.])], 
np.array([  0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,
         4.5,   5. ,   5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,
         9. ,   9.5,  10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,
        13.5,  14. ,  14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,
        18. ,  18.5]))
gate = a[0][0]
print(gate)
print(sum(gate[2:])/5)

toccsl = a[0][1]
print(sum(toccsl[2:])/5)

In [ ]:
# Creates movie from provided csv file
import numpy as np
import photogate as pg
import matplotlib.pyplot as plt
import time
import csv
import os
from matplotlib import animation

# for debugging only: 
%load_ext autoreload  
%autoreload 2

plt.rcParams['animation.ffmpeg_path'] = 'C:\\Program Files\\ffmpeg\\bin\\ffmpeg.exe'
#dirout = os.path.join(os.getcwd(), 'output', 'size_scan_no_gate')
dirout = 'E:\\Work\\Data\\Simulation results\\2D photogate\\WITH TRACKING DATA\\05D_TIRF_TOCCSL_trials'
file = 'data_trial_0'

file_reader, data_file, params, fig, ax = pg.init_anim_plot(dirout, file)

# set up images for animation
ims = []
i = 0
plot_nth_frame = 1
last_frame = 100000

for row in file_reader:
    
    if i%plot_nth_frame == 0 and i < last_frame:
        artists = pg.plot_anim_frame(row, params, ax, i) 
        ims.append(artists)
    
    if i%50 == 0:
        print('step '+str(i))
    i += 1  
    
data_file.close()

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.ArtistAnimation(fig, ims, interval=20, blit=True)

# Save the animation as an mp4.  This requires ffmpeg or mencoder to be installed
mywriter = animation.FFMpegWriter(fps=30, bitrate=3000)
save_file_name = os.path.join(dirout, 'video', file+'.mp4')
start_time = time.time()
anim.save(save_file_name, writer=mywriter, extra_args=['-vcodec', 'libx264'])
print("Finished writing animation in", time.time() - start_time, "seconds")

del mywriter, anim

plt.close('all')
print('Finished writing video')

The following code snippet demonstrates how  the intensity profile of a ring made by sweeping a 2D Gaussian beam is calculated. It numerically evaluates the integral $$I(d)=\int_{0}^{2\pi} a e^{-\frac{R^2+d^2-2Rdcos\theta}{2c^2}} d\theta,$$ 
where I is the intensity at the point of interest, $\theta$ is the angular position along the ring (the integral 
sweeps around the entire ring to calculate the intensity contribution of every point), R is the radius of the ring, 
d is the distance of the point of interest from the center of the ring, a is the amplitude of the Gaussian beam and 
c is the width of the Gaussian beam.

The $R^2+d^2-2Rdcos\theta$ in the numerator appears directly from the law of cosines.

To reduce computational cost, the integral is not computed for every point in the 2D space individually. 
Rather, to exploit the radial symmetry of the problem, the integral is evaluated along a vector of d values 
from zero to the maximum desired value, with the spacing of elements within the vector set by num_steps. Then, 
to obtain the approximate intensity at any given point (just a meshgrid array in this cell), the radius is computed 
for each point and the closest pre-calculated intensity value is used.

In [ ]:
import photogate as pg
import math
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib

# override any default properties in FluorescenceProfile class:
fluo_props = {'num_steps' : 1000,
              'r_ring' : 8,
              'gauss_width' : 0.4,
              'box_width' : 15}
            
profile = pg.FluorescenceProfile(**fluo_props)

# Build 2D heatmap
start_time = time.time()

X,Y,Z = profile.build_heat_map(500)

print("Finished building heatmap in", time.time() - start_time, "seconds")

plt.figure('heat_map')
plt.pcolormesh(X,Y,Z)
plt.colorbar()
plt.show()



In [ ]:
# plot data from result files
import numpy as np
import os
import photogate as pg
# for debugging only: 
%load_ext autoreload  
%autoreload 2

save_result, plot_result = True, True

base_dir = 'E:\\Work\\Data\\Simulation results\\2D photogate'

curr_dir = 'diffusion_consts_20kSteps_noGate'
result_name, var_name = 'density', 'diffusion_const'

target_dir = os.path.join(base_dir, curr_dir)

data = pg.make_result_heatmap(target_dir, result_name, var_name)

pg.plot_result_heatmap(data, save_result=True, disp_result=True, 
                        file_ext='png', zlim = (1e-1,1e2), color_map='gist_ncar')


In [ ]:
"""Make svg heat map plots out of pickled x,y,z files"""

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import photogate as pg
import pickle
import os

base_dir = 'E:\\Work\\Manuscripts\\2D photogate\\' + \
            'Figures\\simulation results\\source data'
color_map = 'gist_ncar'

save_dir = os.path.join(base_dir, color_map)
if not os.path.exists(save_dir):
            os.makedirs(save_dir)

result_files = [x for x in os.listdir(base_dir) if os.path.splitext(x)[1] == '.pickle']


for file in result_files:
    with open(os.path.join(base_dir, file), 'rb') as f:
        obj = pickle.load(f)
    if len(obj) == 3:
        x, y, z = obj[:]
    else:
        info, x, y, z = obj[:]
    
    name = os.path.splitext(file)[0]  
    fig1 = plt.figure(name)
    z = z + 0.0001
    plt.pcolormesh(x,y,z, 
                   norm=LogNorm(vmin=1e-1, vmax=1e2),
                   cmap=color_map)
    plt.colorbar()
    file_name = name + '.svg'
    file_path = os.path.join(save_dir, file_name)
    fig1.savefig(file_path)
    plt.close('all')

In [ ]:
"""Make a single x-y plot from a pickled file"""

import matplotlib.pyplot as plt
import photogate as pg
import pickle
import os

base_dir = 'E:\\Work\\Manuscripts\\2D photogate\\' + \
            'Figures\\simulation results\\source data'

file = 'gate_freq_DiffC1_density_vs_freq.pickle'

with open(os.path.join(base_dir, file), 'rb') as f:
    info, t, var, val = pickle.load(f)

time_point = 15

name = os.path.splitext(file)[0]  

fig, ax = plt.subplots()
ax.set_title('At time '+str(t[time_point]) + ' s')
ax.set_xlabel('Gate frequency, Hz')
ax.set_ylabel('Density of fluorophores in ROI')
ax.set_xlim([0,2])
ax.semilogy(var, val[:,time_point])
plt.show()

file_name = name + '.png'
file_path = os.path.join(base_dir, file_name)
fig.savefig(file_path)
    